In [ ]:
import csv
import json
from typing import TypedDict
from collections import defaultdict

In [ ]:
file = "../../data/toflit18_all_flows.csv"

In [ ]:
class DataPoint(TypedDict):
    product: str 
    import_dk: int
    import_dkb: int
    export_dk: int
    export_dkb: int

In [ ]:
report_dk = defaultdict(dict)
report_others = defaultdict(dict)

with open(file, "r") as muerte:
    reader = csv.DictReader(muerte)
    for i, row in enumerate(reader):
        if row["year"] == "1789":
            if row["export_import"] == "Imports" and row["partner"] == "Dunkerque":
                if row["product_RE_aggregate"]:
                    value = float(row["value"]) if row["value"] else 0.0
                    if row["customs_region_grouping"] not in report_others[row["product_RE_aggregate"]]:
                        report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]] = {}
                    report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]]["product"] = row["product_RE_aggregate"]
                    report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]]["region"] = row["customs_region_grouping"]
                    report_others[row["product_RE_aggregate"]][row["customs_region_grouping"]]["import"] = report_dk[row["product_RE_aggregate"]].get("import", 0) + value

            if row["customs_region_grouping"] == "Flandre" and row["customs_office"] in ['Dunkerque', 'Dunkerque-Basseville']:
                if row["product_RE_aggregate"]:
                    value = float(row["value"]) if row["value"] else 0.0
                    report_dk[row["product_RE_aggregate"]]["product"] = row["product_RE_aggregate"]
                    if row["customs_office"] == 'Dunkerque':
                        if row["export_import"] == "Exports":
                            report_dk[row["product_RE_aggregate"]]["export_dk"] = report_dk[row["product_RE_aggregate"]].get("export_dk", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[row["product_RE_aggregate"]]["import_dk"] = report_dk[row["product_RE_aggregate"]].get("import_dk", 0) + value
                    elif row["customs_office"] == 'Dunkerque-Basseville':
                        if row["export_import"] == "Exports":
                            report_dk[row["product_RE_aggregate"]]["export_dkb"] = report_dk[row["product_RE_aggregate"]].get("export_dkb", 0) + value
                        elif row["export_import"] == "Imports":
                            report_dk[row["product_RE_aggregate"]]["import_dkb"] = report_dk[row["product_RE_aggregate"]].get("import_dkb", 0) + value

In [ ]:
report_others.keys()

In [ ]:
with open("data/report_dunkerque.json", "w") as f:
    json.dump(report_dk, f)

In [ ]:
with open("data/report_other_regions.json", "w") as f:
    json.dump(report_others, f)